In [15]:
import os

import requests
import dask.dataframe as dd
from datetime import datetime
import json
import pandas as pd

from dotenv import load_dotenv


import cm_utlity_library as cm_utility

load_dotenv()

True

In [16]:
# Test token retrieval
cm_access_token = cm_utility.get_cm_access_token()
if cm_access_token:
    print("CM token successfully retrieved")
else:
    print("Token retrieval failed")

CM token successfully retrieved


In [17]:
facility_id_list = [i for i in range(1000, 1004)] # 1000 to 1003

In [63]:

residency_list = []

for facility_id in facility_id_list:
    print(f"Processing facility: {facility_id}")
    try:
        df = cm_utility.get_residency(facility_id, cm_access_token)
        df["FacilityId"] = facility_id
        residency_list.append(df)
    except Exception as e:
        print(f"Error processing facility {facility_id}: {e}")


Processing facility: 1000
Processing facility: 1001
Processing facility: 1002
Processing facility: 1003


Filter only active resident

In [64]:
df_residency = pd.concat(residency_list, ignore_index=True)
df_residency = df_residency[df_residency["DateTerminated"].isna()]
df_residency = df_residency[["Id", "ResidentId", "FacilityId"]]
USER_ID = '2905' # For user cmapi

In [58]:
df_residency.head()

,Id,ResidentId,FacilityId
0,5,4,1000
1,53,7,1000
2,11,11,1000
3,206,17,1000
4,432,19,1000


Get all forms

In [31]:
headers = {
    "Authorization": f"Bearer {cm_access_token}",
    "Accept": "application/json"
}

In [104]:
def fetch_api(url, headers):
    resp = requests.get(url, headers=headers)
    
    try:
        resp.raise_for_status()
        return resp.json()
    except requests.exceptions.HTTPError as e:
        try:
            error_details = resp.json()
        except ValueError:
            error_details = resp.text

        raise requests.HTTPError(
            f"API request failed: {resp.status_code} - {error_details}"
        ) from e

In [118]:
USER_ID = 2905
form_list = []

for _, row in df_residency.iterrows():
    all_form_url = (
        f"{cm_utility.BASE_URL}/api/AssessmentForms/GetResidentForms"
        f"?residentId={row['ResidentId']}"
        f"&facilityId={row['FacilityId']}"
        f"&status=1&pageSize=100"
        f"&userId={USER_ID}"
    )

    try:
        forms = fetch_api(all_form_url, headers=headers)
        # print(f"Got {len(forms)} forms for ResidentId {row['ResidentId']}") # For debugging

        for form in forms:
            form_record = {
                "ResidentId": row["ResidentId"],
                "FacilityId": row["FacilityId"],
                "form": form  # raw full JSON
            }
            form_list.append(form_record)

    except requests.HTTPError as e:
        print(f"Error fetching data for ResidentId {row['ResidentId']}: {e}")


In [123]:
df_forms_raw = pd.DataFrame(form_list)
df_forms_raw["formId"] = df_forms_raw["form"].apply(lambda f: f.get("id"))

In [124]:
df_forms_raw.head()

,ResidentId,FacilityId,form,formId
0,4,1000,"{'id': 108725, 'name': 'Resident of the Day - ...",108725
1,7,1000,"{'id': 156383, 'name': 'Accident / Incident Re...",156383
2,7,1000,"{'id': 146139, 'name': 'Allied Health Referral...",146139
3,7,1000,"{'id': 145552, 'name': 'Resident of the Day - ...",145552
4,7,1000,"{'id': 44000, 'name': 'Allied Health Referral ...",44000


Get form detail

In [125]:
def flatten_form_detail(form_json):
    # Extract top-level metadata safely
    form_id = form_json.get("formId") or form_json.get("id")
    template = form_json.get("template", {})
    template_name = template.get("templateName") or template.get("name")
    form_status = form_json.get("status")
    items = form_json.get("items") or []

    if not items:
        print(f"Form {form_id} has no items.")
        return pd.DataFrame()

    flat_rows = []

    for item in items:
        # Handle missing question structure flexibly
        question_block = item.get("templateQuestion") or item
        question_id = question_block.get("id") or item.get("questionId")
        question_text = question_block.get("text") or item.get("questionText")
        question_type = question_block.get("responseType") or item.get("questionType")
        question_order = question_block.get("order") or item.get("questionOrder")

        responses = item.get("responses") or []

        # No responses → still record the question as a row with null values
        if not responses:
            flat_rows.append({
                "formId": form_id,
                "templateName": template_name,
                "formStatus": form_status,
                "questionId": question_id,
                "questionText": question_text,
                "questionOrder": question_order,
                "responseType": question_type,
                "responseId": None,
                "stringValue": None,
                "booleanValue": None,
                "numericValue": None,
                "dateValue": None,
                "userFirst": None,
                "userLast": None,
                "userRole": None,
                "responseDate": None
            })
            continue

        # Process each actual response
        for r in responses:
            r_info = r.get("recordInformation") or {}
            flat_rows.append({
                "formId": form_id,
                "templateName": template_name,
                "formStatus": form_status,
                "questionId": question_id,
                "questionText": question_text,
                "questionOrder": question_order,
                "responseType": question_type,
                "responseId": r.get("responseId"),
                "stringValue": r.get("stringValue"),
                "booleanValue": r.get("booleanValue"),
                "numericValue": r.get("numericValue"),
                "dateValue": r.get("dateValue"),
                "userFirst": r_info.get("userNameFirst"),
                "userLast": r_info.get("userNameLast"),
                "userRole": r_info.get("userQualification"),
                "responseDate": r_info.get("dateCreated")
            })

    if flat_rows:
        print(f"Flattened formId {form_id} with {len(flat_rows)} rows.")
    else:
        print(f"No responses extracted from formId {form_id}")

    return pd.DataFrame(flat_rows)


In [129]:
form_detail_list = []

for _, row in df_forms_raw.iterrows():
    form_id = row["formId"]

    form_detail_url = (
        f"{cm_utility.BASE_URL}/api/AssessmentForms/GetAssessmentFormForEdit"
        f"?formId={form_id}&userId={USER_ID}"
    )

    try:
        form_detail = fetch_api(form_detail_url, headers=headers)
        form_detail_list.append({
            "formId": form_id,
            "ResidentId": row.get("ResidentId"),
            "FacilityId": row.get("FacilityId"),
            "form": form_detail
        })

    except Exception as e:
        print(f"Failed to fetch form detail for formId {form_id}: {e}")


In [130]:
df_form_details = pd.DataFrame(form_detail_list)

In [133]:
display(df_form_details)

,formId,ResidentId,FacilityId,form
0,108725,4,1000,"{'formId': 108725, 'formVersion': 2, 'status':..."
1,156383,7,1000,"{'formId': 156383, 'formVersion': 2, 'status':..."
2,146139,7,1000,"{'formId': 146139, 'formVersion': 0, 'status':..."
3,145552,7,1000,"{'formId': 145552, 'formVersion': 2, 'status':..."
4,44000,7,1000,"{'formId': 44000, 'formVersion': 0, 'status': ..."
...,...,...,...,...
1120,152410,1236,1003,"{'formId': 152410, 'formVersion': 0, 'status':..."
1121,152135,1236,1003,"{'formId': 152135, 'formVersion': 0, 'status':..."
1122,154381,1241,1003,"{'formId': 154381, 'formVersion': 0, 'status':..."
1123,154329,1241,1003,"{'formId': 154329, 'formVersion': 1, 'status':..."


In [131]:
flattened_forms = []

for _, row in df_form_details.iterrows():
    df_flat = flatten_form_detail(row["form"])
    df_flat["ResidentId"] = row["ResidentId"]
    df_flat["FacilityId"] = row["FacilityId"]
    flattened_forms.append(df_flat)

df_all_flattened = pd.concat(flattened_forms, ignore_index=True)


Flattened formId 108725 with 15 rows.
Flattened formId 156383 with 25 rows.
Form 146139 has no items.
Flattened formId 145552 with 16 rows.
Form 44000 has no items.
Form 43895 has no items.
Form 132614 has no items.
Form 80126 has no items.
Flattened formId 102870 with 18 rows.
Form 149851 has no items.
Flattened formId 148111 with 27 rows.
Form 148188 has no items.
Flattened formId 128078 with 16 rows.
Form 32079 has no items.
Flattened formId 26731 with 21 rows.
Flattened formId 24784 with 23 rows.
Flattened formId 17359 with 24 rows.
Flattened formId 967 with 18 rows.
Form 100396 has no items.
Flattened formId 29611 with 11 rows.
Flattened formId 155252 with 25 rows.
Flattened formId 155297 with 16 rows.
Flattened formId 121365 with 16 rows.
Flattened formId 87065 with 16 rows.
Flattened formId 155983 with 41 rows.
Flattened formId 155971 with 54 rows.
Flattened formId 155996 with 41 rows.
Flattened formId 155991 with 54 rows.
Flattened formId 156570 with 16 rows.
Form 13599 has no 

C:\Users\HarryZhan\AppData\Local\Temp\ipykernel_19780\2841955266.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_all_flattened = pd.concat(flattened_forms, ignore_index=True)


In [132]:
display(df_all_flattened)

,formId,templateName,formStatus,questionId,questionText,questionOrder,responseType,responseId,stringValue,booleanValue,numericValue,dateValue,userFirst,userLast,userRole,responseDate,ResidentId,FacilityId
0,108725.0,Resident of the Day - Registered Nurse Review,0.0,1811.0,RESIDENT OF THE DAY,1.0,5.0,NaN,None,None,NaN,2024-11-15T00:00:00Z,Skye,Jones,Registered Nurse,2024-11-15T01:29:32.0966667Z,4,1000
1,108725.0,Resident of the Day - Registered Nurse Review,0.0,1810.0,"Read progress notes, including GP and other th...",2.0,3.0,NaN,None,True,NaN,None,Skye,Jones,Registered Nurse,2024-11-15T02:51:04.4Z,4,1000
2,108725.0,Resident of the Day - Registered Nurse Review,0.0,1812.0,Read the extended care plan; meet with the res...,3.0,3.0,NaN,None,True,NaN,None,Skye,Jones,Registered Nurse,2024-11-15T02:51:04.4Z,4,1000
3,108725.0,Resident of the Day - Registered Nurse Review,0.0,1813.0,Give direction to PC to ensure they complete t...,4.0,3.0,NaN,None,True,NaN,None,Skye,Jones,Registered Nurse,2024-11-15T01:29:32.0966667Z,4,1000
4,108725.0,Resident of the Day - Registered Nurse Review,0.0,1814.0,"Complete a full set of observations, record th...",5.0,1.0,NaN,None,None,NaN,None,None,None,None,None,4,1000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20653,154329.0,Communication and sensory engagement assessment,0.0,5354.0,"Additional strategies to support independence,...",76.0,6.0,NaN,None,None,NaN,None,None,None,None,None,1241,1003
20654,154329.0,Communication and sensory engagement assessment,0.0,5355.0,Does the resident have any specific preference...,77.0,6.0,NaN,None,None,NaN,None,None,None,None,None,1241,1003
20655,154329.0,Communication and sensory engagement assessment,0.0,771.0,Are there any risks associated with the reside...,78.0,6.0,NaN,None,None,NaN,None,None,None,None,None,1241,1003
20656,154329.0,Communication and sensory engagement assessment,0.0,5356.0,What are the strategies in place to minimise r...,79.0,6.0,NaN,None,None,NaN,None,None,None,None,None,1241,1003


TODO:

1. Strip off the below to another notebook and possibly do it in the bricks
2. Add loading to SQL Server

In [134]:
def map_to_medication_incident_schema(df_flat, facility_id=None):
    df = df_flat.copy()

    # Normalize question text
    df["NormalizedQuestion"] = df["questionText"].str.strip().str.lower()

    # Combine all possible answer types into one field
    df["Answer"] = (
        df["stringValue"]
        .fillna(df["booleanValue"])
        .fillna(df["dateValue"])
        .fillna(df["numericValue"])
    )

    # Rename columns to match the view
    df.rename(columns={
        "formId": "ClinicalAssessmentFormId",
        "responseDate": "CreatedDateUtc",
        "responseId": "CareReceiverId",
        "userFirst": "CareGiverFirst",
        "userLast": "CareGiverLast",
        "userRole": "CareGiverRole",
        "userId": "CareGiverId",
        "questionText": "OriginalQuestion",
        "formStatus": "FormStatus",
        "formVersion": "VersionNumber"
    }, inplace=True)

    # Add external fields if not in df
    if "FacilityId" not in df.columns and facility_id is not None:
        df["FacilityId"] = facility_id

    # Ensure required columns exist
    for col in ["AssessmentFormTemplateId", "CompletedDateUtc", "DeletedDateUtc"]:
        if col not in df.columns:
            df[col] = None  # fill with default nulls

    # Reorder columns to match view schema
    desired_columns = [
        "FacilityId",
        "AssessmentFormTemplateId",
        "ClinicalAssessmentFormId",
        "CareReceiverId",
        "CareGiverId",
        "CreatedDateUtc",
        "CompletedDateUtc",
        "DeletedDateUtc",
        "VersionNumber",
        "NormalizedQuestion",
        "Answer"
    ]

    return df[desired_columns]


In [ ]:
df_responses = pd.concat(
    [flatten_form_detail(form) for form in form_detail_list],
    ignore_index=True
)
df_responses.head()

,formId,templateName,formStatus,questionId,questionText,questionOrder,responseType,responseId,stringValue,booleanValue,numericValue,dateValue,userFirst,userLast,userRole,responseDate
0,155959,Accident / Incident Report,0,10,PART 1 - DETAILS OF INCIDENT,1,3,NaN,No,False,None,None,None,None,None,None
1,155959,Accident / Incident Report,0,2387,Is this incident classified as Serious Incident?,2,1,36648.0,None,None,None,None,Shatwika,Aryal,Registered Nurse,2025-07-27T20:22:46.93Z
2,155959,Accident / Incident Report,0,11,Date of incident,3,5,NaN,None,None,None,2025-07-28T00:00:00Z,Shatwika,Aryal,Registered Nurse,2025-07-27T20:22:46.93Z
3,155959,Accident / Incident Report,0,12,Time of incident,4,2,NaN,06:15,None,None,None,Shatwika,Aryal,Registered Nurse,2025-07-27T20:22:46.93Z
4,155959,Accident / Incident Report,0,13,Site of incident,5,1,36649.0,None,None,None,None,Shatwika,Aryal,Registered Nurse,2025-07-27T20:22:46.93Z


In [ ]:
# Map to medication view schema
df_medication = map_to_medication_incident_schema(df_responses, facility_id=123)

KeyError: "['CareGiverId', 'VersionNumber'] not in index"